In [12]:
import sympy as sp
import numpy as np
from sympy import sin, cos, atan2, diag, Matrix, zeros, simplify, diff

# Start timing
import time
start_time = time.time()

# Define symbolic variables
h, r, m, g, h2 = sp.symbols('h r m g h2')  # height, arm radius, arm mass fraction, gravity, arm height
yaw, pit, rol, q4 = sp.symbols('yaw pit rol q4')  # coordinates (rad)
yawd, pitd, rold, q4d = sp.symbols('yawd pitd rold q4d')  # velocities (rad/s)
Isx, Isy, Isz = sp.symbols('Isx Isy Isz')  # shell inertia
Iax, Iay, Iaz = sp.symbols('Iax Iay Iaz')  # arm inertia

Is = diag(Isx, Isy, Isz)  # shell inertia tensor
Ia = diag(Iax, Iay, Iaz)  # arm inertia tensor

q = Matrix([yaw, pit, rol, q4])  # coordinate vector
qd = Matrix([yawd, pitd, rold, q4d])  # velocity vector

# Forward kinematics
R01 = Matrix([[cos(yaw), -sin(yaw), 0],
              [sin(yaw), cos(yaw), 0],
              [0, 0, 1]])
d01 = Matrix([0, 0, 0])

R12 = Matrix([[cos(pit), 0, sin(pit)],
              [0, 1, 0],
              [-sin(pit), 0, cos(pit)]])
d12 = Matrix([0, 0, 0])

R23 = Matrix([[1, 0, 0],
              [0, cos(rol), -sin(rol)],
              [0, sin(rol), cos(rol)]])
d23 = Matrix([0, -h * sin(rol), h * cos(rol)])

R34 = Matrix([[cos(q4), -sin(q4), 0],
              [sin(q4), cos(q4), 0],
              [0, 0, 1]])
d34 = Matrix([r * cos(q4), r * sin(q4), h2])

# Homogeneous transformation matrices
H01 = R01.row_join(d01).col_join(Matrix([0, 0, 0, 1]).T)
H12 = R12.row_join(d12).col_join(Matrix([0, 0, 0, 1]).T)
H23 = R23.row_join(d23).col_join(Matrix([0, 0, 0, 1]).T)
H34 = R34.row_join(d34).col_join(Matrix([0, 0, 0, 1]).T)

R02 = R01 * R12
R03 = R02 * R23
R04 = R03 * R34

H02 = H01 * H12
H03 = H02 * H23
H04 = H03 * H34

CG = H04 * Matrix([-r * 1/(1+m), 0, -h2 * 1/(1+m), 1])  # CG location
pts = [Matrix([0, 0, 0, 1]), H03[:, 3], H03 * Matrix([0.3 * r, 0, 0, 1]),
       H03[:, 3], CG, H04[:, 3]]

# Differential kinematics
def diffT(T, vars, order):
    """Compute time derivative of vector T."""
    n = len(vars) // 2
    q = vars[:n]  # joint positions
    qd = vars[n:]  # joint velocities

    T = Matrix(T)
    q = Matrix(q)
    qd = Matrix(qd)
    
    # Compute Jacobian and first derivative
    J = T.jacobian(q)  # Use Matrix.jacobian method
    dT = J * qd  # dT/dt = ∂T/∂q * dq/dt
    
    # Higher-order derivatives
    if order > 1:
        for _ in range(order - 1):
            dT = diffT(dT, vars, 1)
    return dT

z00 = Matrix([0, 0, 1])
y01 = R01[:, 1]
x02 = R02[:, 0]
z03 = R03[:, 2]

J1v = zeros(3, 4)
J2v = zeros(3, 4)
J3v = Matrix.hstack(z00.cross(H03[:3, 3]),
                    y01.cross(H03[:3, 3]),
                    x02.cross(H03[:3, 3]),
                    zeros(3, 1))
J4v = Matrix.hstack(z00.cross(H04[:3, 3]),
                    y01.cross(H04[:3, 3]),
                    x02.cross(H04[:3, 3]),
                    z03.cross(H04[:3, 3]) - H03[:3, 3])

J1w = Matrix.hstack(z00, zeros(3, 3))
J2w = Matrix.hstack(z00, y01, zeros(3, 2))
J3w = Matrix.hstack(z00, y01, x02, zeros(3, 1))
J4w = Matrix.hstack(z00, y01, x02, z03)

v_CG = diffT(CG[:3], Matrix([q, qd]), 1)
a_CG = diffT(v_CG, Matrix([q, qd]), 1)
F = a_CG + Matrix([0, 0, 9.81])

# Potential energy
P = 1 * g * H03[2, 3] + m * g * H04[2, 3]  # Potential energy (J)
G = P.jacobian(q).T  # Use Matrix.jacobian method

# M & C matrices
M = 1 * (J3v.T * J3v) + (J3w.T * R03 * Is * R03.T * J3w) + \
    m * (J4v.T * J4v) + (J4w.T * R04 * Ia * R04.T * J4w)

K = 0.5 * qd.T * M * qd
E = P + K[0]  # K is a 1x1 matrix, extract scalar

C = zeros(4, 4)
for k in range(4):
    for j in range(4):
        for i in range(4):
            C[k, j] += 0.5 * (diff(M[k, j], q[i]) +
                             diff(M[k, i], q[j]) -
                             diff(M[i, j], q[k])) * qd[i]
C = simplify(C)

# External forces
J_CG = CG[:3].jacobian(q)  # Use Matrix.jacobian method

# Display timing
elapsed_time = time.time() - start_time
print(f"symbolics in {elapsed_time:.2f}s")

TypeError: can't multiply sequence by non-int of type 'MutableDenseMatrix'

In [11]:
CG

Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw) + h2*(sin(pit)*cos(rol)*cos(yaw) + sin(rol)*sin(yaw)) - h2*(sin(pit)*cos(rol)*cos(yaw) + sin(rol)*sin(yaw))/(m + 1) + r*(sin(pit)*sin(rol)*cos(yaw) - sin(yaw)*cos(rol))*sin(q4) + r*cos(pit)*cos(q4)*cos(yaw) - r*((sin(pit)*sin(rol)*cos(yaw) - sin(yaw)*cos(rol))*sin(q4) + cos(pit)*cos(q4)*cos(yaw))/(m + 1)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw) + h2*(sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw)) - h2*(sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw))/(m + 1) + r*(sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*sin(q4) + r*sin(yaw)*cos(pit)*cos(q4) - r*((sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*sin(q4) + sin(yaw)*cos(pit)*cos(q4))/(m + 1)],
[                                                                                                                                                                            h*cos(pit)*cos(rol) + h2*cos(pit)*cos(rol) - h2*cos(pit)*cos(rol)/(m + 1) - r*sin(pit)*cos(q4) + r

In [10]:
Matrix(CG[:3]).jacobian

Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw) + h2*(sin(pit)*cos(rol)*cos(yaw) + sin(rol)*sin(yaw)) - h2*(sin(pit)*cos(rol)*cos(yaw) + sin(rol)*sin(yaw))/(m + 1) + r*(sin(pit)*sin(rol)*cos(yaw) - sin(yaw)*cos(rol))*sin(q4) + r*cos(pit)*cos(q4)*cos(yaw) - r*((sin(pit)*sin(rol)*cos(yaw) - sin(yaw)*cos(rol))*sin(q4) + cos(pit)*cos(q4)*cos(yaw))/(m + 1)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw) + h2*(sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw)) - h2*(sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw))/(m + 1) + r*(sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*sin(q4) + r*sin(yaw)*cos(pit)*cos(q4) - r*((sin(pit)*sin(rol)*sin(yaw) + cos(rol)*cos(yaw))*sin(q4) + sin(yaw)*cos(pit)*cos(q4))/(m + 1)],
[                                                                                                                                                                            h*cos(pit)*cos(rol) + h2*cos(pit)*cos(rol) - h2*cos(pit)*cos(rol)/(m + 1) - r*sin(pit)*cos(q4) + r